In [291]:

using LinearAlgebra
using GenericLinearAlgebra


In [292]:
function complete_orthonormal(U::Matrix{Float64})
    d, r = size(U)
    r ≤ d || error("U deve ter no máximo d colunas")

    r == d && return zeros(Float64, d, 0)   # já completa

    # Projeta vetores aleatórios no complemento de span(U)
    Z = randn(Float64, d, d - r)
    Z .-= U * (U' * Z)                      # remove componente em U

    # QR fino garante ortonormalidade
    C = Matrix(qr(Z).Q)                  # d × (d−r)
    return C
end

complete_orthonormal (generic function with 1 method)

In [293]:
function build_csd_input_matrix(D1::Matrix{Float64}, D2::Matrix{Float64})
    # Dimensões
    m, d = size(D1)  # D1: m x d
    n, _ = size(D2)  # D2: n x d

    # Passo 1: QR de D1^T para obter base de R(D1) em R^d
    Q1 = Matrix(qr(D1').Q)  # Q1 é d x m
    R1 = Matrix(qr(D1').R)  # R1 é m x m

    println("Q1: ", size(Q1))  # Debugging: verificar tamanho de Q1
    println("R1: ", size(R1))  # Debugging: verificar tamanho de R1

    r1 = sum(abs.(diag(R1)) .> eps() * maximum(abs.(diag(R1))))  # Estimar posto
    E1 = Q1[:, 1:r1]  
    E2 = complete_orthonormal(E1)  # Base ortonormal completa para R^d, d x (d - r1)
    E = hcat(E1, E2)  # Base ortonormal completa para R^d, d x d
    
    # Passo 2: QR de D2^T para obter base de R(D2) em R^d
    Q2 = Matrix(qr(D2').Q)  # Q2 é d x n
    R2 = Matrix(qr(D2').R)  # R2 é n x n

    println("Q2: ", size(Q2))  # Debugging: verificar tamanho de Q2
    println("R2: ", size(R2))  # Debugging: verificar tamanho de R2

    c1 = sum(abs.(diag(R2)) .> eps() * maximum(abs.(diag(R2))))  # Estimar posto
    F1 = Q2[:, 1:c1]  
    F2 = complete_orthonormal(F1)  # Base ortonormal completa para R^d, d x (d - c1)
    

    # Passo 3: Calcular Q11 = E1^T F1
    Q11 = E1' * F1  # r1 x c1
    Q12 = E1' * F2  # r1 x (d - c1)
    Q21 = E2' * F1  # (d - r1) x c1
    Q22 = E2' * F2  # (d - r1) x (d - c1)

    # Passo 4: Concatenar Q11, Q12, Q21, Q22
    Qtop = hcat(Q11, Q12)
    Qbot = hcat(Q21, Q22)
    Q = vcat(Qtop, Qbot)
    
    return Q, r1, c1, E, E1, F1, R1, R2
end

build_csd_input_matrix (generic function with 1 method)

In [294]:
# Exemplo com dados aleatórios
n = 10  # amostras dos datasets
d = 50  # número de features

D1 = randn(n, d)  # dataset 1
D2 = randn(n, d)  # dataset 2

Q, r1, c1, E, E1, F1, R1, R2 = build_csd_input_matrix(D1, D2)

Q1: (50, 10)
R1: (10, 10)
Q2: (50, 10)
R2: (10, 10)


([0.05757245502941141 -0.07196626902948877 … 0.04079125044842743 0.012685959408021315; 0.1544531653437436 -0.2245030549746553 … -0.0706288661924742 -0.08037899842005644; … ; 0.302672615103471 0.3208591357428787 … -0.11404880388828759 0.13287315553651932; 0.04975533343232731 0.036193371568234614 … 0.16767191133009338 -0.058433733831021], 10, 10, [-0.17532902833657094 -0.11376122488441215 … 0.18858031959928198 -0.04609197681322293; 0.11129230601318629 0.0020358116890105693 … -0.016962002297662336 0.09352360302224018; … ; 0.0660739654012759 0.1985982581312741 … 0.05171661895569457 -0.029366991755677252; -0.08933978221691871 -0.03076502158231202 … 0.2692439095399951 -0.236460743382251], [-0.17532902833657094 -0.11376122488441215 … -0.07937360656417568 0.044231138702821235; 0.11129230601318629 0.0020358116890105693 … -0.049396378890456144 -0.18012355183095777; … ; 0.0660739654012759 0.1985982581312741 … -0.24882107117258728 0.2353278407603851; -0.08933978221691871 -0.03076502158231202 … -0.

In [295]:
display(size(Q))  # Verifica o tamanho da matriz Q
println("Posto de D1: ", r1)
println("Posto de D2: ", c1)
println("Tamanho de E: ", size(E))  # Verifica o tamanho da base ortonormal completa E
println("Tamanho de E1: ", size(E1))  # Verifica o tamanho de E1
println("Tamanho de F1: ", size(F1))  # Verifica o tamanho de E2
println("Tamanho de R1: ", size(R1))  # Verifica o tamanho de R1
println("Tamanho de R2: ", size(R2))  # Verifica o tamanho de R2

(50, 50)

Posto de D1: 10
Posto de D2: 10
Tamanho de E: (50, 50)
Tamanho de E1: (50, 10)
Tamanho de F1: (50, 10)
Tamanho de R1: (10, 10)
Tamanho de R2: (10, 10)


In [296]:
# Verificar se R1' * E1' é aproximadamente igual a D1
if R1'*E1' ≈ D1
    println("R1' * E1' é aproximadamente igual a D1")
else
    println("R1' * E1' é diferente de D1")
end

R1' * E1' é aproximadamente igual a D1


In [297]:
function rebuild_datasets_matrices(E, R)
    D_reconstructed = R' * E'  
    # Retornar a matriz reconstruída
    return D_reconstructed
end

rebuild_datasets_matrices (generic function with 1 method)

In [298]:
function verify_Q(Q::Matrix{Float64})
    if Q'*Q ≈ I
        println("A matriz Q é ortonormal")
    else
        println("A matriz Q não é ortonormal")
        
    end
end

verify_Q (generic function with 1 method)

In [299]:
# Exibir e verificar a corretude da matriz A
verify_Q(Q)
D1_reconstructed = rebuild_datasets_matrices(E1, R1)
D2_reconstructed = rebuild_datasets_matrices(F1, R2)

if D1_reconstructed ≈ D1 && D2_reconstructed ≈ D2
    println("As matrizes reconstruídas são equivalentes às originais!")
else
    println("As matrizes reconstruídas NÃO são equivalentes às originais.")
        
end

A matriz Q é ortonormal
As matrizes reconstruídas são equivalentes às originais!


In [300]:
function csd(Q, c1, r1, E)
    d= size(Q, 1)  # Dimensão do espaço
    # separar Q em blocos
    Q11 = Q[1:r1, 1:c1]  # r1 x c1
    Q12 = Q[1:r1, c1+1:end]  # r1 x (d - c1)
    Q21 = Q[r1+1:end, 1:c1]  # ( d - r1) x c1
    Q22 = Q[r1+1:end, c1+1:end]  # (d - r1) x (d - c1)

    # SVD dos blocos de Q = EᵀF
    U1, S11, V1 = svd(Q11; full=true)     
    U2, S22, V2  = svd(Q22; full=true)

    D11 = U1' * Q11 * V1
    D12 = U1' * Q12 * V2 
    D21 = U2' * Q21 * V1
    D22 = U2' * Q22 * V2
    
    # Monta matriz diagonalizada D via blocos projetados
    D_top = hcat(D11, D12)  # r1 x d
    D_bot = hcat(D21, D22)  # (d - r1) x (d - c1)
    D = vcat(D_top, D_bot)  # d × d

    # Constrói matriz U unificada
    U = Matrix{Float64}(I, d, d)
    U[1:r1, 1:r1] = U1
    U[r1+1:end, r1+1:end] = U2
    
    # Constrói matriz V unificada
    V= Matrix{Float64}(I, d, d)
    V[1:c1, 1:c1] = V1
    V[c1+1:end, c1+1:end] = V2
    
    # Calcula os ângulos θ entre os subespaços
    θ_rad = acos.(clamp.(S11, -1.0, 1.0))
    θ_deg = θ_rad .* (180 / π)

    # Reconstrói a rotação W que leva E1 → F1
    W = E * U * D * U' * E'


    return U, U1, U2, V, V1, V2, D, θ_deg, W  
end

csd (generic function with 2 methods)

In [301]:
U, U1, U2, V, V1, V2, D, θ, W =csd(Q, c1, r1, E)

([0.20821539267536313 0.185097314389168 … 0.0 0.0; -0.40717571861997937 0.2880021378730248 … 0.0 0.0; … ; 0.0 0.0 … -0.4197734413302249 0.44361450905679334; 0.0 0.0 … 0.06723143586609646 0.11276056713480694], [0.20821539267536313 0.185097314389168 … 0.18269110357676321 -0.5405636947286412; -0.40717571861997937 0.2880021378730248 … -0.31572231034903725 0.2330741179553038; … ; 0.37682859922890743 -0.018686929313946006 … -0.09048293393559602 0.16379788601199438; -0.5489328842281799 -0.5837143213181492 … 0.28347138121211374 -0.195858718809661], [0.12449030926634644 -0.10302430707613792 … 0.19062568533592714 0.08505789766289765; -0.15916264727343804 -0.01516367675046056 … -0.19467621054061207 -0.34936833331190276; … ; 0.050414778746165664 -0.14564078318429935 … -0.4197734413302249 0.44361450905679334; -0.040516416291134325 -0.3039732571943635 … 0.06723143586609646 0.11276056713480694], [-0.164273091990983 0.40802475039488595 … 0.0 0.0; 0.1566643232726775 -0.07364653204351264 … 0.0 0.0; … ; 

In [302]:
println("Ângulos principais entre subespaços (em radianos): ", θ)
println("tamanho de U: ", size(U))
println("tamanho de U1: ", size(U1))
println("tamanho de U2: ", size(U2))
println("tamanho de V: ", size(V))
println("tamanho de V1: ", size(V1))
println("tamanho de V2: ", size(V2))
println("tamanho de D: ", size(D))
println("tamanho de W: ", size(W))

Ângulos principais entre subespaços (em radianos): [45.25743409218133, 51.7638492779527, 52.593671191846354, 62.22242235404505, 65.44462302394899, 69.8941410192203, 77.52446534577948, 82.39607395130436, 84.03389920374514, 89.57581596454811]
tamanho de U: (50, 50)
tamanho de U1: (10, 10)
tamanho de U2: (40, 40)
tamanho de V: (50, 50)
tamanho de V1: (10, 10)
tamanho de V2: (40, 40)
tamanho de D: (50, 50)
tamanho de W: (50, 50)


In [306]:
display(θ)                 # Ângulos principais entre subespaços (em radianos)
@show(U*D*V' ≈ Q) 
display(norm(W-I) ≈ norm(D-I))  # Deve ser próximo de 0 (rotação direta correta)
@show W*E1*U1*V1' ≈ F1 

10-element Vector{Float64}:
 45.25743409218133
 51.7638492779527
 52.593671191846354
 62.22242235404505
 65.44462302394899
 69.8941410192203
 77.52446534577948
 82.39607395130436
 84.03389920374514
 89.57581596454811

U * D * V' ≈ Q = true
W * E1 * U1 * V1' ≈ F1 = true


true

true

In [304]:
D1_reconstructed = rebuild_datasets_matrices(E1, R1)
D2_reconstructed = rebuild_datasets_matrices(F1, R2)

if D1_reconstructed ≈ D1 && D2_reconstructed ≈ D2
    println("As matrizes reconstruídas são equivalentes às originais!")
else
    println("As matrizes reconstruídas NÃO são equivalentes às originais.")
        
end

As matrizes reconstruídas são equivalentes às originais!
